In [1]:
##### add python path #####
import sys
import os
PATH = os.getcwd()
for dir_idx, dir_name in enumerate(PATH.split('/')):
    if 'optimal_control' in dir_name.lower():
        PATH = '/'.join(PATH.split('/')[:(dir_idx+1)])
        break
if not PATH in sys.path:
    sys.path.append(PATH)
###########################

from gait_scheduler import GaitScheduler
from env.quadruped.env import Env

from copy import deepcopy
import numpy as np
import time

In [2]:
env = Env(enable_draw=True, base_fix=True)
num_leg = env.num_leg
time_step = env.time_step

gs_args = dict()
gs_args['num_leg'] = num_leg
gs_args['gait_period'] = 1.0
gs_args['is_visualize'] = True
gait_scheduler = GaitScheduler(gs_args)

In [3]:
env.reset()

array([ 0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -0.1,
       -0.7,  1.5,  0.1, -0.7,  1.5, -0.1, -0.7,  1.5,  0.1, -0.7,  1.5,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ])

In [ ]:
pre_contact_list = []
swing_trajectory_list = [[]]*num_leg

global_time = 0.0
while global_time < 10.0:

    if global_time >= 1.0:
        gait_list = np.zeros((num_leg, 2)) # FR, FL, RR, RL
        gait_list[0,0] = 0.5 # offset
        gait_list[0,1] = 0.5 # duration
        gait_list[2,0] = 0.0 # offset
        gait_list[2,1] = 0.5 # duration
        gait_period = 1.0
        gait_scheduler.set_gait_list(gait_list, gait_period)

    contact_list, gait_info_list = gait_scheduler.step(time_step)
    if len(pre_contact_list) != 0:
        for leg_idx in range(num_leg):
            if pre_contact_list[leg_idx] == 1 and contact_list[leg_idx] == 0:
                swing_period, stand_period, swing_elapsed_time = gait_info_list[leg_idx]
                p_init = env.model.foot_pos_list[leg_idx]
                base_p_init = env.model.base_foot_pos_list[leg_idx]
                ref_base_pos = env.model.base_com_pos + env.model.abduct_org_list[leg_idx] + env.model.thigh_org_list[leg_idx] \
                + Ts*0.5*base_target_vel + np.sqrt(target_x[2,0]/9.8)*(base_com_vel - base_target_vel)
                    ref_base_pos[2] = z_c
                
                
        p0, p_ref, ts, te = swing_trajectory
        z0, zc = p0[2], p_ref[2]
        desired_base_foot_pos = (p_ref - p0)*(elapsed_t - ts)/(te - ts) + p0
        desired_base_foot_pos[2] = (z0 - zc)*(((2*elapsed_t - ts - te)/(te - ts))**2) + zc
        desired_base_foot_vel = (p_ref - p0)/(te - ts)
        desired_base_foot_vel[2] = 4*(z0 - zc)*(2*elapsed_t - ts - te)/((te - ts)**2)
        desired_base_foot_acc[2] = 8*(z0 - zc)/((te - ts)**2)
        pass
    pre_contact_list = deepcopy(contact_list)
    
    force_list = np.zeros((num_leg, 3))
    contact_list = np.ones(num_leg)
    env.step(force_list, contact_list)

    #time.sleep(time_step)
    global_time += time_step
    
gait_scheduler.close()
env.close()